**Подключение к базе данных**

В .env файле указывается строка подключения к базе данных PostgreSQL (см. .env.example). Данная строка используется в качестве параметра при инициализации класса для работы с БД (Database).

In [1]:
from database import Database
from dotenv import load_dotenv, find_dotenv
from os import getenv
import logging.config

load_dotenv(find_dotenv())

logging_config = {
    'version': 1,
    'disable_existing_loggers': False,

    'formatters': {
        'default_formatter': {
            'format': '[%(levelname)s] %(message)s'
        },
    },

    'handlers': {
        'stream_handler': {
            'class': 'logging.StreamHandler',
            'formatter': 'default_formatter',
        },
    },

    'loggers': {
        'my_logger': {
            'handlers': ['stream_handler'],
            'level': 'INFO',
            'propagate': False
        }
    }
}

logging.config.dictConfig(logging_config)
logger = logging.getLogger('my_logger')

db = Database(getenv('DATABASE_URL'), logger)

[INFO] a connection to the database has been established


Получение списка публикаций пользователя Instagram.

*Замечание: информация о публикациях должна быть предварительно собрана с использованием InstagramScraper и загружена в БД (DATABASE_URL).*

In [2]:
posts = db.get_user_posts('https://www.instagram.com/russkih_aleksey/')
for post in posts[:5]:
    print(f'URL поста: {post.url}\n'
          f'Содержание поста: {post.text}\n'
          f'Время публикации поста: {post.time}\n'
          f'Хеш-теги поста: {post.tags}\n'
          f'Упоминания в посте: {post.links}',
          end='\n\n')

URL поста: https://www.instagram.com/p/CX097AWNTm2/
Содержание поста: С большим вниманием смотрел пресс-конференцию Президента России Владимира Владимировича Путина. Глава государства затронул много важных тем. 

🔹ИНСТРУМЕНТЫ ДЛЯ РОСТА ЭКОНОМИКИ 

Президент обозначил, какие есть драйверы для экономического роста в условиях пандемии, какие существуют инструменты поддержки на федеральном уровне. Наша задача активно входить во все федеральные программы и улучшать инфраструктуру в регионе. В этом году Ульяновская область получила мощную поддержку от Правительства РФ на развитие индустриальных парков. А это новые предприятия, новые рабочие места для людей. 

🔹ОБЕСПЕЧЕНИЕ ИНФРАСТРУКТУРЫ ДЛЯ ЗЕМЕЛЬНЫХ УЧАСТКОВ МНОГОДЕТНЫХ

Ещё одна актуальная для Ульяновской области тема - обеспечение качественной инфраструктурой земельных участков для многодетных семей. Особенно остро проблема стоит в крупнейших городах региона – Ульяновске и Димитровграде. 

Важно, что Президент обратил внимание на то, что 

Получение списка комментариев к публикациям пользователя Instagram.

*Замечание: информация о комментариях должна быть предварительно собрана с использованием InstagramScraper  и загружена в БД (DATABASE_URL).*

In [3]:
comments = db.get_user_posts_comments('https://www.instagram.com/russkih_aleksey/')
for comment in comments[:5]:
    print(f'URL комментария: {comment.url}\n'
          f'URL автора комментария: {comment.owner_url}\n'
          f'Содержание комментария: {comment.text}\n'
          f'Время публикации комментария: {comment.time}\n'
          f'Хеш-теги комментария: {comment.tags}\n'
          f'Упоминания в комментарии: {comment.links}',
          end='\n\n')

URL комментария: https://www.instagram.com/p/CX097AWNTm2/c/18220123078096586/
URL автора комментария: https://www.instagram.com/sepsiz86/
Содержание комментария: Алексей Юрьевич,что насчёт канализации на амурской думаете делать,как будем решать проблему?
Время публикации комментария: 2021-12-23 10:30:24+00:00
Хеш-теги комментария: []
Упоминания в комментарии: []

URL комментария: https://www.instagram.com/p/CX097AWNTm2/c/17912435033110175/
URL автора комментария: https://www.instagram.com/olga_elka86/
Содержание комментария: Как быть тем многодетным которых вообще не ставят на очередь на землю из за того что один ребёнок на территории папы прописан???Глупые критерии!
Время публикации комментария: 2021-12-23 10:39:19+00:00
Хеш-теги комментария: []
Упоминания в комментарии: []

URL комментария: https://www.instagram.com/p/CX097AWNTm2/c/17903516834289944/
URL автора комментария: https://www.instagram.com/valnatal1969/
Содержание комментария: Остановите ценыыыыыыыы!!!!!!!! Скоро по меру по

Подключение пакета для анализа текстов, [srsparser](https://github.com/Text-Analysis/srsparser), который содержит методы предобработки текста, извлечения ключевых слов и получения TF-IDF весов слов.

In [4]:
%pip install srsparser --upgrade

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\f0rge\PycharmProjects\posts-scraper\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Извлечение из списка полученных комментариев по пользователю содержимого комментариев, длина которых больше 20 символов, убрав из содержания упоминания пользователей Instagram.

In [5]:
filtered_comments = [Database.remove_links_from_comment(comment)
                     for comment in comments
                     if len(comment.text) > 20]

comments_contents = [comment.text for comment in filtered_comments]

Инициализация класса, ответствнного за обработку текста на естественном языке (включает в себя методы извлечения ключевых слов с использованием pullenti, построение tf-idf модели с использованием gensim и др.)

In [6]:
from srsparser import NLProcessor

nlp = NLProcessor(init_pullenti=True)

Построение на основе отобранного содержимого комментариев TF-IDF модели и получение весов слов в этмх комментариях. В качестве примера осуществляется вывод первых 10 элементов.

In [7]:
comments_tf_idf_weights = nlp.get_tf_idf_weights(comments_contents)

n = 10 if len(comments_tf_idf_weights) >= 10 else len(comments_tf_idf_weights)
for i in range(n):
    print('Комментарий:', comments_contents[i])
    print('TF-IDF веса слов в комментарии:', comments_tf_idf_weights[i], end='\n\n')

Комментарий: Алексей Юрьевич,что насчёт канализации на амурской думаете делать,как будем решать проблему?
TF-IDF веса слов в комментарии: [['амурской', 0.527], ['насчёт', 0.428], ['решать', 0.341], ['канализация', 0.332], ['думать', 0.304], ['делать', 0.242], ['быть', 0.204], ['проблема', 0.204], ['юриевич', 0.199], ['алексей', 0.197]]

Комментарий: Как быть тем многодетным которых вообще не ставят на очередь на землю из за того что один ребёнок на территории папы прописан???Глупые критерии!
TF-IDF веса слов в комментарии: [['глупый', 0.394], ['критерий', 0.382], ['папа', 0.372], ['прописать', 0.372], ['земля', 0.303], ['многодетный', 0.296], ['ставить', 0.29], ['очередь', 0.203], ['территория', 0.203], ['вообще', 0.176], ['который', 0.149], ['ребёнок', 0.148]]

Комментарий: Остановите ценыыыыыыыы!!!!!!!! Скоро по меру пойдём!😢
TF-IDF веса слов в комментарии: [['ценыыыыыыы', 0.598], ['остановить', 0.472], ['поидти', 0.454], ['скоро', 0.379], ['мера', 0.264]]

Комментарий: Соглашусь Ост

Выделение из каждого комментария ключевых слов (или словосочетаний) на основе отобранного содержимого комментариев. В качестве примера осуществляется вывод первых 10 элементов.

In [8]:
comments_keywords = []
for i in range(len(comments_contents)):
    try:
        content_keywords = nlp.get_keywords_pullenti(comments_contents[i])
        comments_keywords.append(content_keywords)
    except Exception:
        comments_keywords.append([])

n = 10 if len(comments_keywords) >= 10 else len(comments_keywords)
for i in range(n):
    print('Комментарий:', comments_contents[i])
    print('Ключевые слова в комментарии:', comments_keywords[i], end='\n\n')

Комментарий: Алексей Юрьевич,что насчёт канализации на амурской думаете делать,как будем решать проблему?
Ключевые слова в комментарии: ['ЮРЬЕВИЧ', 'КАНАЛИЗАЦИЯ', 'ДУМАТЬ', 'ДЕЛАТЬ', 'РЕШАТЬ', 'ПРОБЛЕМА']

Комментарий: Как быть тем многодетным которых вообще не ставят на очередь на землю из за того что один ребёнок на территории папы прописан???Глупые критерии!
Ключевые слова в комментарии: ['ТЕРРИТОРИЯ ПАПЫ', 'ГЛУПЫЙ КРИТЕРИЙ', 'СТАВИТЬ', 'ОЧЕРЕДЬ', 'ЗЕМЛЯ', 'РЕБЕНОК', 'ТЕРРИТОРИЯ', 'ПАПА', 'ПРОПИСАТЬ', 'ГЛУПЫЙ', 'КРИТЕРИЙ']

Комментарий: Остановите ценыыыыыыыы!!!!!!!! Скоро по меру пойдём!😢
Ключевые слова в комментарии: ['ОСТАНОВИТЬ', 'МЕРА', 'ПОЙТИ']

Комментарий: Соглашусь Остановите цены сегодня зашла с ума сойти капуста 50 руб килограмм в Гулливере сахар за 50 руб перевалил 900 г просто беспредел хлеб дорожает основные виды продуктов, которые мы покупаем это первая необходимость и она недоступна со мной думаю согласятся все особенно пенсионеры🔥🔥🔥
Ключевые слова в комментарии: ['О

**Связывание TF-IDF весов слов комментариев с ключевыми словами, извлечёнными из комментариев.**

Рассматривается ключевое слово (или словосочетание) и ему в соответствие ставится TF-IDF вес (если для такого слова он был посчитан). TF-IDF вес для словосочетаний рассчитывается как сумма TF-IDF весов слов, из которых состоит словосочетание. В качестве примера осуществляется вывод первых 10 элементов.

In [9]:
# словарь, в котором в качестве ключа выступает id комментария в БД, а в качестве значения - список ключевых слов комментария с указанием их весов tf-idf
comments_with_tf_idf_keywords = {}
# проход по ключевым словам всех комментариев (список, в котором каждый элемент представлен списком ключевых слов комментария)
for i in range(len(comments_keywords)):
    # словарь, в котором хранятся пары вида "ключевое слово - tf-idf вес" для i-го комментария
    comment_keywords_with_ratios = {}
    # проход по ключевым словам i-того комментария (список, в котором каждый элемент представлен либо ключевым словом, либо ключевым словосочетанием)
    for keyword in comments_keywords[i]:
        # проверка, элемент является словом или словосочетанием
        keyphrase_words = keyword.split()
        if len(keyphrase_words) > 1:  # словосочетание
            tf_idf_sum = 0.0
            # проход по словам ключевого словосочетания
            for word in keyphrase_words:
                # лемматизируем слово (получаем его нормальную форму)
                normal_word = nlp.get_normal_form(word)
                for tf_idf_weight in comments_tf_idf_weights[i]:
                    # если для ключевого слова нашёлся его tf-idf вес
                    if normal_word.lower() == tf_idf_weight[0]:
                        tf_idf_sum += tf_idf_weight[1]
                        break
            if tf_idf_sum > 0.0:
                comment_keywords_with_ratios[keyword] = tf_idf_sum
        else:  # слово
            # лемматизируем слово (получаем его нормальную форму)
            normal_word = nlp.get_normal_form(keyword)
            # проход по tf-idf весам слов всех комментариев (список, в котором каждый элемент представлен списком пар "слово-вес")
            for tf_idf_weight in comments_tf_idf_weights[i]:
                # если для ключевого слова нашёлся его tf-idf вес
                if normal_word.lower() == tf_idf_weight[0]:
                    comment_keywords_with_ratios[keyword] = tf_idf_weight[1]
                    break
    # если для ключевых слов комментария нашлись tf-idf веса, то ставим пары "ключевое слово - tf-idf вес" в соответствие комментарию
    if len(comment_keywords_with_ratios) > 0:
        comments_with_tf_idf_keywords[filtered_comments[i].id] = dict(
            sorted(comment_keywords_with_ratios.items(), key=lambda item: item[1], reverse=True)
        )

n = 10 if len(comments_with_tf_idf_keywords) >= 10 else len(comments_with_tf_idf_keywords)
c = 0
for comment_id, comment_keywords in comments_with_tf_idf_keywords.items():
    if c == n:
        break

    print('Комментарий:', db.get_comment(comment_id).text)
    print('Ключевые слова:', comment_keywords, end='\n\n')
    c += 1

Комментарий: Алексей Юрьевич,что насчёт канализации на амурской думаете делать,как будем решать проблему?
Ключевые слова: {'РЕШАТЬ': 0.341, 'КАНАЛИЗАЦИЯ': 0.332, 'ДУМАТЬ': 0.304, 'ДЕЛАТЬ': 0.242, 'ПРОБЛЕМА': 0.204, 'ЮРЬЕВИЧ': 0.199}

Комментарий: Как быть тем многодетным которых вообще не ставят на очередь на землю из за того что один ребёнок на территории папы прописан???Глупые критерии!
Ключевые слова: {'ГЛУПЫЙ КРИТЕРИЙ': 0.776, 'ТЕРРИТОРИЯ ПАПЫ': 0.575, 'ГЛУПЫЙ': 0.394, 'КРИТЕРИЙ': 0.382, 'ПАПА': 0.372, 'ПРОПИСАТЬ': 0.372, 'ЗЕМЛЯ': 0.303, 'СТАВИТЬ': 0.29, 'ОЧЕРЕДЬ': 0.203, 'ТЕРРИТОРИЯ': 0.203, 'РЕБЕНОК': 0.148}

Комментарий: Остановите ценыыыыыыыы!!!!!!!! Скоро по меру пойдём!😢
Ключевые слова: {'ОСТАНОВИТЬ': 0.472, 'МЕРА': 0.264}

Комментарий: Соглашусь Остановите цены сегодня зашла с ума сойти капуста 50 руб килограмм в Гулливере сахар за 50 руб перевалил 900 г просто беспредел хлеб дорожает основные виды продуктов, которые мы покупаем это первая необходимость и она недоступна со м

Применение функции [softmax](https://www.helenkapatsa.ru/softmaks/) к ключевым словам каждого комментария с целью нормализации tf-idf весов ключевых слов. В качестве примера осуществляется вывод первых 10 элементов.

In [10]:
import numpy as np

for comment_id, comment_keywords in comments_with_tf_idf_keywords.items():
    # перебор каждой степени i и вычисление экспонент
    exps = [np.exp(i) for i in comment_keywords.values()]
    sum_of_exps = sum(exps)
    softmax = [j / sum_of_exps for j in exps]

    i = 0
    for key in comment_keywords.keys():
        comment_keywords[key] = softmax[i]
        i += 1

n = 10 if len(comments_with_tf_idf_keywords) >= 10 else len(comments_with_tf_idf_keywords)
c = 0
for comment_id, comment_keywords in comments_with_tf_idf_keywords.items():
    if c == n:
        break

    print('Комментарий:', db.get_comment(comment_id).text)
    print('Ключевые слова:', comment_keywords, end='\n\n')
    c += 1

Комментарий: Алексей Юрьевич,что насчёт канализации на амурской думаете делать,как будем решать проблему?
Ключевые слова: {'РЕШАТЬ': 0.17856981172498693, 'КАНАЛИЗАЦИЯ': 0.17696989384933356, 'ДУМАТЬ': 0.17208346605314348, 'ДЕЛАТЬ': 0.16173830484304486, 'ПРОБЛЕМА': 0.15570755911020503, 'ЮРЬЕВИЧ': 0.15493096441928625}

Комментарий: Как быть тем многодетным которых вообще не ставят на очередь на землю из за того что один ребёнок на территории папы прописан???Глупые критерии!
Ключевые слова: {'ГЛУПЫЙ КРИТЕРИЙ': 0.13496037017453388, 'ТЕРРИТОРИЯ ПАПЫ': 0.11038576453286196, 'ГЛУПЫЙ': 0.09210978502288764, 'КРИТЕРИЙ': 0.09101107305890879, 'ПАПА': 0.09010549775130604, 'ПРОПИСАТЬ': 0.09010549775130604, 'ЗЕМЛЯ': 0.08409786507302035, 'СТАВИТЬ': 0.08301166840265514, 'ОЧЕРЕДЬ': 0.07609489509500823, 'ТЕРРИТОРИЯ': 0.07609489509500823, 'РЕБЕНОК': 0.07202268804250371}

Комментарий: Остановите ценыыыыыыыы!!!!!!!! Скоро по меру пойдём!😢
Ключевые слова: {'ОСТАНОВИТЬ': 0.5518133302331701, 'МЕРА': 0.4481866697